BBC, The Guardian, The Mirror, The Independent, The Times, The Telegraph, The Sun, The Daily Express, The Daily Mail


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import time
import random
import pandas as pd

# BBC

## BBC Immigration

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 5

# create lists to store our data
all_news_links = []
all_headlines = []
all_dates = []

for page_number in range(5, 14):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    bbc = f"https://www.bbc.co.uk/news/topics/c302m85qe1vt?page={page_number}"
    
    # request webpage
    res = requests.get(bbc, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("div", class_="ssrcss-1ns4t85-PromoSwitchLayoutAtBreakpoints et5qctl0")

    for article in articles:
        # check if the article date contains the word "Video" or "Audio"
        article_date = article.find("span", class_="visually-hidden ssrcss-1f39n02-VisuallyHidden e16en2lz0").get_text().strip()
        if "Video" in article_date or "Audio" in article_date:
            continue  # Skip this article and move to the next one
        
        # append article link
        article_info = article.find("a", class_="ssrcss-1mrs5ns-PromoLink exn3ah91")
        link = "https://www.bbc.co.uk" + article_info.attrs["href"]
        all_news_links.append(link)

        # append headline
        headline = article.find("p", class_="ssrcss-15dlehh-PromoHeadline exn3ah96").get_text().strip()
        all_headlines.append(headline)

        # append date
        all_dates.append(article_date)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"Headline": all_headlines,
        "Links": all_news_links,
        "Date": all_dates
}
df = pd.DataFrame.from_dict(data)
# write to csv
df.to_csv(r"bbc-uk-immigration.csv", encoding="utf-8", header="true", index=False)


In [ ]:
# reload csv
bbc_immigration = pd.read_csv("bbc-uk-immigration.csv")

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# function to get body of article
def get_body(df, retries=3):
    bodies = []
    for link in df['Links']:
        for attempt in range(retries):
            try:
                res = requests.get(link, headers=headers)
                res.raise_for_status()  # raise an HTTPError for bad responses
                soup = bs(res.content, 'lxml')
                main_content = soup.find_all("div", {"class": "ssrcss-uf6wea-RichTextComponentWrapper ep2nwvo0"})

                # extract text from the text-block
                article_text = ' '.join(content.get_text() for content in main_content).strip()
                
                # append the body text
                bodies.append(article_text)
                break  # break the retry loop on success
            
            except (requests.exceptions.RequestException, requests.exceptions.SSLError) as e:
                print(f"Error fetching {link}: {e}")
                if attempt < retries - 1:
                    wait_time = 2 ** attempt  # exponential backoff
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    print(f"Failed to fetch {link} after {retries} attempts.")
                    bodies.append("")  # append an empty string if all retries fail

    # add the bodies list as a new column to the DataFrame
    df['Body'] = bodies

    return df

df = get_body(bbc_immigration)

# rewrite to CSV
df.to_csv(r"bbc-uk-immigration.csv", encoding="utf-8", header=True, index=False)



## BBC Migration

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 4

# create lists to store our data
all_news_links = []
all_headlines = []
all_dates = []

for page_number in range(4, 16):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    bbc = f"https://www.bbc.co.uk/news/topics/cz4pr2gdg1et?page={page_number}"

    
    # request webpage
    res = requests.get(bbc, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("div", class_="ssrcss-1ns4t85-PromoSwitchLayoutAtBreakpoints et5qctl0")

    for article in articles:
        # check if the article date contains the word "Video" or "Audio"
        article_date = article.find("span", class_="visually-hidden ssrcss-1f39n02-VisuallyHidden e16en2lz0").get_text().strip()
        if "Video" in article_date or "Audio" in article_date:
            continue  # Skip this article and move to the next one
        
        # append article link
        article_info = article.find("a", class_="ssrcss-1mrs5ns-PromoLink exn3ah91")
        link = "https://www.bbc.co.uk" + article_info.attrs["href"]
        all_news_links.append(link)

        # append headline
        headline = article.find("p", class_="ssrcss-15dlehh-PromoHeadline exn3ah96").get_text().strip()
        all_headlines.append(headline)

        # append date
        all_dates.append(article_date)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"Headline": all_headlines,
        "Links": all_news_links,
        "Date": all_dates
}
df = pd.DataFrame.from_dict(data)
# write to csv
df.to_csv(r"bbc-uk-migration.csv", encoding="utf-8", header="true", index=False)


In [ ]:
# reload csv
bbc_migration = pd.read_csv("bbc-uk-migration.csv")

df = get_body(bbc_migration)

# rewrite to CSV
df.to_csv(r"bbc-uk-migration.csv", encoding="utf-8", header=True, index=False)


## BBC Visas

In [ ]:
# create lists to store our data
all_news_links = []
all_headlines = []
all_dates = []

# define our user headers
headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
bbc = f"https://www.bbc.co.uk/news/topics/c1m1wly10gzt?page=1"

# request webpage
res = requests.get(bbc, headers=headers)

# check status for debugging
res.raise_for_status()

soup = bs(res.text, "html.parser")

# this gets list of articles
articles = soup.find_all("div", class_="ssrcss-1ns4t85-PromoSwitchLayoutAtBreakpoints et5qctl0")

for article in articles:
    # check if the article date contains the word "Video" or "Audio"
    article_date = article.find("span", class_="visually-hidden ssrcss-1f39n02-VisuallyHidden e16en2lz0").get_text().strip()
    if "Video" in article_date or "Audio" in article_date:
        continue  # Skip this article and move to the next one
    
    # append article link
    article_info = article.find("a", class_="ssrcss-1mrs5ns-PromoLink exn3ah91")
    link = "https://www.bbc.co.uk" + article_info.attrs["href"]
    all_news_links.append(link)

    # append headline
    headline = article.find("p", class_="ssrcss-15dlehh-PromoHeadline exn3ah96").get_text().strip()
    all_headlines.append(headline)

    # append date
    all_dates.append(article_date)

time.sleep(random.randint(1, 3))


# Convert data to dataframe
data = {"Headline": all_headlines,
    "Links": all_news_links,
    "Date": all_dates
}
df = pd.DataFrame.from_dict(data)
# write to csv
df.to_csv(r"bbc-uk-visas.csv", encoding="utf-8", header="true", index=False)

In [ ]:
# reload csv
bbc_visas = pd.read_csv("bbc-uk-visas.csv")

df = get_body(bbc_visas)

# rewrite to CSV
df.to_csv(r"bbc-uk-visas.csv", encoding="utf-8", header=True, index=False)


## BBC Refugees and asylum seekers

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 9

# create lists to store our data
all_news_links = []
all_headlines = []
all_dates = []

for page_number in range(9, 35):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    bbc = f" https://www.bbc.co.uk/news/topics/cg41ylwvxmdt?page={page_number}"
    
    # request webpage
    res = requests.get(bbc, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("div", class_="ssrcss-1ns4t85-PromoSwitchLayoutAtBreakpoints et5qctl0")

    for article in articles:
        # check if the article date contains the word "Video" or "Audio"
        article_date = article.find("span", class_="visually-hidden ssrcss-1f39n02-VisuallyHidden e16en2lz0").get_text().strip()
        if "Video" in article_date or "Audio" in article_date:
            continue  # Skip this article and move to the next one
        
        # append article link
        article_info = article.find("a", class_="ssrcss-1mrs5ns-PromoLink exn3ah91")
        link = "https://www.bbc.co.uk" + article_info.attrs["href"]
        all_news_links.append(link)

        # append headline
        headline = article.find("p", class_="ssrcss-15dlehh-PromoHeadline exn3ah96").get_text().strip()
        all_headlines.append(headline)

        # append date
        all_dates.append(article_date)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"Headline": all_headlines,
        "Links": all_news_links,
        "Date": all_dates
}
df = pd.DataFrame.from_dict(data)
# write to csv
df.to_csv(r"bbc-uk-refugees.csv", encoding="utf-8", header="true", index=False)


In [ ]:
# reload csv
bbc_refugees = pd.read_csv("bbc-uk-refugees.csv")

df = get_body(bbc_refugees)

# rewrite to CSV
df.to_csv(r"bbc-uk-refugees.csv", encoding="utf-8", header=True, index=False)


## Merging all the dataframes


In [ ]:
# load csvs
bbc_refugees = pd.read_csv("bbc-uk-refugees.csv")
bbc_visas = pd.read_csv("bbc-uk-visas.csv")
bbc_migration = pd.read_csv("bbc-uk-migration.csv")
bbc_immigration = pd.read_csv("bbc-uk-immigration.csv")

# combine all dataframes
bbc = pd.concat([bbc_refugees, bbc_visas, bbc_migration, bbc_immigration])

# write to csv
bbc.to_csv(r"all-bbc.csv", encoding="utf-8", header=True, index=False)


# The Guardian

## Section: politics; Tag: uk/immigration

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

API_KEY = "794e32b2-456d-4b24-a0e6-570872ae9488"
page = 1
articles_per_page = 50  # maximum number of articles per page
articles = []

while True:
    url = f"https://content.guardianapis.com/politics?tag=uk/immigration&api-key={API_KEY}&from-date=2023-01-01&to-date=2023-12-31&type=article&page={page}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        current_page_articles = data.get('response', {}).get('results', [])
        
        if not current_page_articles:  # no more articles available
            break
        
        articles.extend(current_page_articles)
        page += 1
    else:
        print(f"Error: {response.status_code} - {response.text}")
        break

with open('guardian-politics-uk-immigration.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['headline', 'link', 'date', 'body']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for article in articles:
        article_url = article['webUrl']
        article_response = requests.get(article_url)
        
        if article_response.status_code == 200:
            soup = BeautifulSoup(article_response.content, 'html.parser')
            div_element = soup.find('div', class_='article-body-commercial-selector article-body-viewer-selector dcr-fp1ya')
            if div_element:
                paragraphs = div_element.find_all('p')
                article_text = ' '.join(p.get_text() for p in paragraphs)
            else:
                article_text = "No content found in the article."
            
            writer.writerow({
                'headline': article['webTitle'],
                'link': article_url,
                'date': article['webPublicationDate'],
                'body': article_text
            })
        else:
            print(f"Error retrieving article from {article_url}: {article_response.status_code}")



## Section: commentisfree; Tag: uk/immigration

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

API_KEY = "794e32b2-456d-4b24-a0e6-570872ae9488"
page = 1
articles_per_page = 50  # maximum number of articles per page
articles = []

while True:
    url = f"https://content.guardianapis.com/commentisfree?tag=uk/immigration&api-key={API_KEY}&from-date=2023-01-01&to-date=2023-12-31&type=article&page={page}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        current_page_articles = data.get('response', {}).get('results', [])
        
        if not current_page_articles:  # no more articles available
            break
        
        articles.extend(current_page_articles)
        page += 1
    else:
        print(f"Error: {response.status_code} - {response.text}")
        break

with open('guardian-commentisfree-uk-immigration.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['headline', 'link', 'date', 'body']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for article in articles:
        article_url = article['webUrl']
        article_response = requests.get(article_url)
        
        if article_response.status_code == 200:
            soup = BeautifulSoup(article_response.content, 'html.parser')
            div_element = soup.find('div', class_='article-body-commercial-selector article-body-viewer-selector dcr-fp1ya')
            if div_element:
                paragraphs = div_element.find_all('p')
                article_text = ' '.join(p.get_text() for p in paragraphs)
            else:
                article_text = "No content found in the article."
            
            writer.writerow({
                'headline': article['webTitle'],
                'link': article_url,
                'date': article['webPublicationDate'],
                'body': article_text
            })
        else:
            print(f"Error retrieving article from {article_url}: {article_response.status_code}")


## Section: uk-news, Tag: global-development/migration



In [21]:
import requests
from bs4 import BeautifulSoup
import csv

API_KEY = "794e32b2-456d-4b24-a0e6-570872ae9488"
page = 1
articles_per_page = 50  # maximum number of articles per page
articles = []

while True:
    url = f"https://content.guardianapis.com/uk-news?tag=global-development/migration&api-key={API_KEY}&from-date=2023-01-01&to-date=2023-12-31&type=article&page={page}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        current_page_articles = data.get('response', {}).get('results', [])
        
        if not current_page_articles:  # no more articles available
            break
        
        articles.extend(current_page_articles)
        page += 1
    else:
        print(f"Error: {response.status_code} - {response.text}")
        break

with open('guardian-uk_news-global_development-migration.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['headline', 'link', 'date', 'body']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for article in articles:
        article_url = article['webUrl']
        article_response = requests.get(article_url)
        
        if article_response.status_code == 200:
            soup = BeautifulSoup(article_response.content, 'html.parser')
            div_element = soup.find('div', class_='article-body-commercial-selector article-body-viewer-selector dcr-fp1ya')
            if div_element:
                paragraphs = div_element.find_all('p')
                article_text = ' '.join(p.get_text() for p in paragraphs)
            else:
                article_text = "No content found in the article."
            
            writer.writerow({
                'headline': article['webTitle'],
                'link': article_url,
                'date': article['webPublicationDate'],
                'body': article_text
            })
        else:
            print(f"Error retrieving article from {article_url}: {article_response.status_code}")


Error: 400 - {"response":{"status":"error","message":"requested page is beyond the number of available pages"}}


## Section: uk-news; Tag: world/refugees

In [22]:
import requests
from bs4 import BeautifulSoup
import csv

API_KEY = "794e32b2-456d-4b24-a0e6-570872ae9488"
page = 1
articles_per_page = 50  # maximum number of articles per page
articles = []

while True:
    url = f"https://content.guardianapis.com/uk-news?tag=world/refugees&api-key={API_KEY}&from-date=2023-01-01&to-date=2023-12-31&type=article&page={page}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        current_page_articles = data.get('response', {}).get('results', [])
        
        if not current_page_articles:  # no more articles available
            break
        
        articles.extend(current_page_articles)
        page += 1
    else:
        print(f"Error: {response.status_code} - {response.text}")
        break

with open('guardian-uk_news-world-refugees.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['headline', 'link', 'date', 'body']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for article in articles:
        article_url = article['webUrl']
        article_response = requests.get(article_url)
        
        if article_response.status_code == 200:
            soup = BeautifulSoup(article_response.content, 'html.parser')
            div_element = soup.find('div', class_='article-body-commercial-selector article-body-viewer-selector dcr-fp1ya')
            if div_element:
                paragraphs = div_element.find_all('p')
                article_text = ' '.join(p.get_text() for p in paragraphs)
            else:
                article_text = "No content found in the article."
            
            writer.writerow({
                'headline': article['webTitle'],
                'link': article_url,
                'date': article['webPublicationDate'],
                'body': article_text
            })
        else:
            print(f"Error retrieving article from {article_url}: {article_response.status_code}")


Error: 400 - {"response":{"status":"error","message":"requested page is beyond the number of available pages"}}


## Merging all the dataframes

In [25]:
import pandas as pd
# load csvs
g1 = pd.read_csv("guardian-politics-uk-immigration.csv")
g2 = pd.read_csv("guardian-commentisfree-uk-immigration.csv")
g3 = pd.read_csv("guardian-uk_news-global_development-migration.csv")
g4 = pd.read_csv("guardian-uk_news-world-refugees.csv")

# combine all dataframes
guardian = pd.concat([g1, g2, g3, g4])

# write to csv
guardian.to_csv(r"all-guardian.csv", encoding="utf-8", header=True, index=False)


# Mirror

## Immigration

In [32]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 1

# create lists to store our data
all_news_links = []

for page_number in range(1, 5):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    if page_number == 1:
        mirror = "https://www.mirror.co.uk/all-about/immigration"
    else:
        mirror = f"https://www.mirror.co.uk/all-about/immigration?pageNumber={page_number}"

    # request webpage
    res = requests.get(mirror, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("article")

    # print out the href attribute of the <a> tag within each article that contains data-link-tracking
    for article in articles:
        # find the <a> tag within the article that contains data-link-tracking
        a_tag = article.find('a', {"data-link-tracking": True})
    
        # extract the href attribute if the <a> tag is found
        if a_tag:
            link = a_tag.get('href')

        all_news_links.append(link)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"link": all_news_links}
df = pd.DataFrame.from_dict(data)

# write to csv
df.to_csv(r"mirror-immigration.csv", encoding="utf-8", header="true", index=False)


We are scraping page: 1
We are scraping page: 2
We are scraping page: 3
We are scraping page: 4


In [35]:
# reload csv
mirror_immigration = pd.read_csv("mirror-immigration.csv")

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# function to get body of article
# function to get body of article
def get_info(df, retries=3):
    all_headlines = []
    all_bodies = []
    all_dates = []
    for link in df['link']:
        for attempt in range(retries):
            try:
                res = requests.get(link, headers=headers)
                res.raise_for_status()  # raise an HTTPError for bad responses
                soup = bs(res.content, 'lxml')

                # extract the headline
                headline_tag = soup.find("h1", class_="lead-content__title")
                if headline_tag:
                    headline = headline_tag.get_text().strip()
                    all_headlines.append(headline)
                else:
                    all_headlines.append("None")

                # extract the date
                time_tag = soup.find("time", class_="date-published")
                if time_tag:
                    date = time_tag.get('datetime')
                    all_dates.append(date)
                else:
                    all_dates.append("None")

                # extract the article body
                div_element = soup.find('div', class_='article-body')
                if div_element:
                    paragraphs = div_element.find_all('p')
                    article_text = ' '.join(p.get_text() for p in paragraphs)
                    all_bodies.append(article_text)
                else:
                    all_bodies.append("None")

                break  # break the retry loop on success
            
            except (requests.exceptions.RequestException, requests.exceptions.SSLError) as e:
                print(f"Error fetching {link}: {e}")
                if attempt < retries - 1:
                    wait_time = 2 ** attempt  # exponential backoff
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    print(f"Failed to fetch {link} after {retries} attempts.")
                    all_headlines.append("None")
                    all_dates.append("None")
                    all_bodies.append("None")

    # add as new column to df
    df['headline'] = all_headlines
    df['date'] = all_dates
    df['body'] = all_bodies

    return df

df = get_info(mirror_immigration)

# rewrite to CSV
df.to_csv(r"mirror-immigration.csv", encoding="utf-8", header=True, index=False)




## Migration

In [39]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 1

# create lists to store our data
all_news_links = []

for page_number in range(1, 3):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    if page_number == 1:
        mirror = "https://www.mirror.co.uk/all-about/migration"
    else:
        mirror = f"https://www.mirror.co.uk/all-about/migration?pageNumber={page_number}"

    # request webpage
    res = requests.get(mirror, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("article")

    # print out the href attribute of the <a> tag within each article that contains data-link-tracking
    for article in articles:
        # find the <a> tag within the article that contains data-link-tracking
        a_tag = article.find('a', {"data-link-tracking": True})
    
        # extract the href attribute if the <a> tag is found
        if a_tag:
            link = a_tag.get('href')

        all_news_links.append(link)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"link": all_news_links}
df = pd.DataFrame.from_dict(data)

# write to csv
df.to_csv(r"mirror-migration.csv", encoding="utf-8", header="true", index=False)

We are scraping page: 1
We are scraping page: 2


In [40]:
# reload csv
mirror_migration = pd.read_csv("mirror-migration.csv")

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

df = get_info(mirror_migration)

# rewrite to CSV
df.to_csv(r"mirror-migration.csv", encoding="utf-8", header=True, index=False)

## Illegal Immigrants

In [41]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 1

# create lists to store our data
all_news_links = []

for page_number in range(1, 4):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    if page_number == 1:
        mirror = "https://www.mirror.co.uk/all-about/illegal-immigrants"
    else:
        mirror = f"https://www.mirror.co.uk/all-about/illegal-immigrants?pageNumber={page_number}"

    # request webpage
    res = requests.get(mirror, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("article")

    # print out the href attribute of the <a> tag within each article that contains data-link-tracking
    for article in articles:
        # find the <a> tag within the article that contains data-link-tracking
        a_tag = article.find('a', {"data-link-tracking": True})
    
        # extract the href attribute if the <a> tag is found
        if a_tag:
            link = a_tag.get('href')

        all_news_links.append(link)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"link": all_news_links}
df = pd.DataFrame.from_dict(data)

# write to csv
df.to_csv(r"mirror-illegal-immigrants.csv", encoding="utf-8", header="true", index=False)

We are scraping page: 1
We are scraping page: 2
We are scraping page: 3


In [42]:
# reload csv
mirror_illegals = pd.read_csv("mirror-illegal-immigrants.csv")

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

df = get_info(mirror_illegals)

# rewrite to CSV
df.to_csv(r"mirror-illegal-immigrants.csv", encoding="utf-8", header=True, index=False)

## Migrant Crisis

In [43]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 1

# create lists to store our data
all_news_links = []

for page_number in range(1, 6):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    if page_number == 1:
        mirror = "https://www.mirror.co.uk/all-about/migrant-crisis"
    else:
        mirror = f"https://www.mirror.co.uk/all-about/migrant-crisis?pageNumber={page_number}"

    # request webpage
    res = requests.get(mirror, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("article")

    # print out the href attribute of the <a> tag within each article that contains data-link-tracking
    for article in articles:
        # find the <a> tag within the article that contains data-link-tracking
        a_tag = article.find('a', {"data-link-tracking": True})
    
        # extract the href attribute if the <a> tag is found
        if a_tag:
            link = a_tag.get('href')

        all_news_links.append(link)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"link": all_news_links}
df = pd.DataFrame.from_dict(data)

# write to csv
df.to_csv(r"mirror-migrant-crisis.csv", encoding="utf-8", header="true", index=False)

We are scraping page: 1
We are scraping page: 2
We are scraping page: 3
We are scraping page: 4
We are scraping page: 5


In [46]:
# reload csv
mirror_crisis = pd.read_csv("mirror-migrant-crisis.csv")

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

df = get_info(mirror_crisis)

# rewrite to CSV
df.to_csv(r"mirror-migrant-crisis.csv", encoding="utf-8", header=True, index=False)

## Home Office

In [47]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 1

# create lists to store our data
all_news_links = []

for page_number in range(7, 21):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    if page_number == 1:
        mirror = "https://www.mirror.co.uk/all-about/home-office"
    else:
        mirror = f"https://www.mirror.co.uk/all-about/home-office?pageNumber={page_number}"

    # request webpage
    res = requests.get(mirror, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("article")

    # print out the href attribute of the <a> tag within each article that contains data-link-tracking
    for article in articles:
        # find the <a> tag within the article that contains data-link-tracking
        a_tag = article.find('a', {"data-link-tracking": True})
    
        # extract the href attribute if the <a> tag is found
        if a_tag:
            link = a_tag.get('href')

        all_news_links.append(link)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"link": all_news_links}
df = pd.DataFrame.from_dict(data)

# write to csv
df.to_csv(r"mirror-home-office.csv", encoding="utf-8", header="true", index=False)

We are scraping page: 7
We are scraping page: 8
We are scraping page: 9
We are scraping page: 10
We are scraping page: 11
We are scraping page: 12
We are scraping page: 13
We are scraping page: 14
We are scraping page: 15
We are scraping page: 16
We are scraping page: 17
We are scraping page: 18
We are scraping page: 19
We are scraping page: 20


In [48]:
# reload csv
mirror_home_office = pd.read_csv("mirror-home-office.csv")

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

df = get_info(mirror_home_office)

# rewrite to CSV
df.to_csv(r"mirror-home-office.csv", encoding="utf-8", header=True, index=False)

## Border Force

In [49]:

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 1

# create lists to store our data
all_news_links = []

for page_number in range(1, 4):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    if page_number == 1:
        mirror = "https://www.mirror.co.uk/all-about/border-force"
    else:
        mirror = f"https://www.mirror.co.uk/all-about/border-force?pageNumber={page_number}"

    # request webpage
    res = requests.get(mirror, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("article")

    # print out the href attribute of the <a> tag within each article that contains data-link-tracking
    for article in articles:
        # find the <a> tag within the article that contains data-link-tracking
        a_tag = article.find('a', {"data-link-tracking": True})
    
        # extract the href attribute if the <a> tag is found
        if a_tag:
            link = a_tag.get('href')

        all_news_links.append(link)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"link": all_news_links}
df = pd.DataFrame.from_dict(data)

# write to csv
df.to_csv(r"mirror-border-force.csv", encoding="utf-8", header="true", index=False)

We are scraping page: 1
We are scraping page: 2
We are scraping page: 3


In [50]:
# reload csv
mirror_border_force = pd.read_csv("mirror-border-force.csv")

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

df = get_info(mirror_home_office)

# rewrite to CSV
df.to_csv(r"mirror-border-force.csv", encoding="utf-8", header=True, index=False)

## Refugee Crisis

In [51]:

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 1

# create lists to store our data
all_news_links = []

for page_number in range(1, 3):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    if page_number == 1:
        mirror = "https://www.mirror.co.uk/all-about/refugee-crisis"
    else:
        mirror = f"https://www.mirror.co.uk/all-about/refugee-crisis?pageNumber={page_number}"

    # request webpage
    res = requests.get(mirror, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("article")

    # print out the href attribute of the <a> tag within each article that contains data-link-tracking
    for article in articles:
        # find the <a> tag within the article that contains data-link-tracking
        a_tag = article.find('a', {"data-link-tracking": True})
    
        # extract the href attribute if the <a> tag is found
        if a_tag:
            link = a_tag.get('href')

        all_news_links.append(link)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"link": all_news_links}
df = pd.DataFrame.from_dict(data)

# write to csv
df.to_csv(r"mirror-refugee-crisis.csv", encoding="utf-8", header="true", index=False)

We are scraping page: 1
We are scraping page: 2


In [52]:
# reload csv
mirror_refugee_crisis = pd.read_csv("mirror-refugee-crisis.csv")

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

df = get_info(mirror_refugee_crisis)

# rewrite to CSV
df.to_csv(r"mirror-refugee-crisis.csv", encoding="utf-8", header=True, index=False)

## Asylum-seekers

In [53]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 1

# create lists to store our data
all_news_links = []

for page_number in range(1, 4):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    if page_number == 1:
        mirror = "https://www.mirror.co.uk/all-about/asylum-seekers"
    else:
        mirror = f"https://www.mirror.co.uk/all-about/asylum-seekers?pageNumber={page_number}"

    # request webpage
    res = requests.get(mirror, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("article")

    # print out the href attribute of the <a> tag within each article that contains data-link-tracking
    for article in articles:
        # find the <a> tag within the article that contains data-link-tracking
        a_tag = article.find('a', {"data-link-tracking": True})
    
        # extract the href attribute if the <a> tag is found
        if a_tag:
            link = a_tag.get('href')

        all_news_links.append(link)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"link": all_news_links}
df = pd.DataFrame.from_dict(data)

# write to csv
df.to_csv(r"mirror-asylum-seekers.csv", encoding="utf-8", header="true", index=False)


We are scraping page: 1
We are scraping page: 2
We are scraping page: 3


In [54]:
# reload csv
mirror_asylum = pd.read_csv("mirror-asylum-seekers.csv")

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

df = get_info(mirror_asylum)

# rewrite to CSV
df.to_csv(r"mirror-asylum-seekers.csv", encoding="utf-8", header=True, index=False)

## Merging all the dataframes

In [55]:
import pandas as pd
# load csvs
m1 = pd.read_csv("mirror-asylum-seekers.csv")
m2 = pd.read_csv("mirror-refugee-crisis.csv")
m3 = pd.read_csv("mirror-border-force.csv")
m4 = pd.read_csv("mirror-home-office.csv")
m5 = pd.read_csv("mirror-migrant-crisis.csv")
m6 = pd.read_csv("mirror-illegal-immigrants.csv")
m7 = pd.read_csv("mirror-migration.csv")
m8 = pd.read_csv("mirror-immigration.csv")



# combine all dataframes
mirror = pd.concat([m1, m2, m3, m4, m5, m6, m7, m8])

# write to csv
mirror.to_csv(r"all-mirror.csv", encoding="utf-8", header=True, index=False)


# Express

## Immigration

In [57]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 1

# create lists to store our data
all_news_links = []

for page_number in range(13, 45):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    if page_number == 1:
        express = "https://www.express.co.uk/latest/immigration"
    else:
        express = f"https://www.express.co.uk/latest/immigration?pageNumber={page_number}"

    # request webpage
    res = requests.get(express, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("li", class_="post")

    # print out the href attribute of the <a> tag within each article that contains data-link-tracking
    for article in articles:
        # find the <a> tag within the article that contains data-link-tracking
        a_tag = article.find('a')
    
        # extract the href attribute if the <a> tag is found
        if a_tag:
            href_tag = a_tag.get('href')
            link = "https://www.express.co.uk" + href_tag

        all_news_links.append(link)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"link": all_news_links}
df = pd.DataFrame.from_dict(data)

# write to csv
df.to_csv(r"express-immigration.csv", encoding="utf-8", header="true", index=False)


We are scraping page: 13
We are scraping page: 14
We are scraping page: 15
We are scraping page: 16
We are scraping page: 17
We are scraping page: 18
We are scraping page: 19
We are scraping page: 20
We are scraping page: 21
We are scraping page: 22
We are scraping page: 23
We are scraping page: 24
We are scraping page: 25
We are scraping page: 26
We are scraping page: 27
We are scraping page: 28
We are scraping page: 29
We are scraping page: 30
We are scraping page: 31
We are scraping page: 32
We are scraping page: 33
We are scraping page: 34
We are scraping page: 35
We are scraping page: 36
We are scraping page: 37
We are scraping page: 38
We are scraping page: 39
We are scraping page: 40
We are scraping page: 41
We are scraping page: 42
We are scraping page: 43
We are scraping page: 44


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Reload CSV
express_immigration = pd.read_csv("express-immigration.csv")

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# Function to get body of article
def get_info(df, retries=5):
    all_headlines = []
    all_dates = []
    all_bodies = []
    for link in df['link']:  
        for attempt in range(retries):
            try:
                res = requests.get(link, headers=headers)
                res.raise_for_status()  # raise an HTTPError for bad responses
                soup = BeautifulSoup(res.content, "html.parser")

                # Extract the headline information
                header_info = soup.find("header", class_="clearfix")

                if header_info:
                    # Get headline
                    headline = header_info.find("h1").get_text().strip()
                    all_headlines.append(headline)
                else:
                    all_headlines.append("None")
                
                # Extract date information
                dates_info = soup.find("div", class_="dates")
                if dates_info:
                    # Extract the first datetime attribute
                    published_time = dates_info.find("time")
                    if published_time and published_time.get("datetime"):
                        published_date = published_time.get("datetime")
                        all_dates.append(published_date)
                    else:
                        all_dates.append("None")
                else:
                    all_dates.append("None")
                    
                # Extract the article body
                article_body = soup.find("div", class_="text-description")
                if article_body:
                    paragraphs = article_body.find_all("p")
                    article_text = "\n".join(p.get_text() for p in paragraphs)
                    all_bodies.append(article_text)
                else:
                    all_bodies.append("None")
                
                # Break the retry loop on success
                break
            
            except requests.exceptions.RequestException as e:
                print(f"Error fetching {link}: {e}")
                if attempt < retries - 1:
                    wait_time = 2 ** attempt  # exponential backoff
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    print(f"Failed to fetch {link} after {retries} attempts.")
                    all_headlines.append("None")
                    all_dates.append("None")
                    all_bodies.append("None")

    # Add as new columns to df
    df['headline'] = all_headlines
    df['date'] = all_dates
    df['body'] = all_bodies

    return df

df = get_info(express_immigration)

# Rewrite to CSV
df.to_csv("express-immigration.csv", encoding="utf-8", header=True, index=False)


## Migrant Crisis

In [7]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import time

# initialise index, this tracks the page number we are on
index = 1

# create lists to store our data
all_news_links = []

for page_number in range(12, 59):
    print(f"We are scraping page: {page_number}")
    # define our user headers
    headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
    if page_number == 1:
        express = "https://www.express.co.uk/latest/migrant-crisis"
    else:
        express = f"https://www.express.co.uk/latest/migrant-crisis?pageNumber={page_number}"

    # request webpage
    res = requests.get(express, headers=headers)
    
    # check status for debugging
    res.raise_for_status()
    
    soup = bs(res.text, "html.parser")

    # this gets list of articles
    articles = soup.find_all("li", class_="post")

    # print out the href attribute of the <a> tag within each article that contains data-link-tracking
    for article in articles:
        # find the <a> tag within the article that contains data-link-tracking
        a_tag = article.find('a')
    
        # extract the href attribute if the <a> tag is found
        if a_tag:
            href_tag = a_tag.get('href')
            link = "https://www.express.co.uk" + href_tag

        all_news_links.append(link)

    time.sleep(random.randint(1, 3))
    

# Convert data to dataframe
data = {"link": all_news_links}
df = pd.DataFrame.from_dict(data)

# write to csv
df.to_csv(r"express-migrant-crisis.csv", encoding="utf-8", header="true", index=False)


We are scraping page: 12
We are scraping page: 13
We are scraping page: 14
We are scraping page: 15
We are scraping page: 16
We are scraping page: 17
We are scraping page: 18
We are scraping page: 19
We are scraping page: 20
We are scraping page: 21
We are scraping page: 22
We are scraping page: 23
We are scraping page: 24
We are scraping page: 25
We are scraping page: 26
We are scraping page: 27
We are scraping page: 28
We are scraping page: 29
We are scraping page: 30
We are scraping page: 31
We are scraping page: 32
We are scraping page: 33
We are scraping page: 34
We are scraping page: 35
We are scraping page: 36
We are scraping page: 37
We are scraping page: 38
We are scraping page: 39
We are scraping page: 40
We are scraping page: 41
We are scraping page: 42
We are scraping page: 43
We are scraping page: 44
We are scraping page: 45
We are scraping page: 46
We are scraping page: 47
We are scraping page: 48
We are scraping page: 49
We are scraping page: 50
We are scraping page: 51


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Reload CSV
express_migrant_crisis = pd.read_csv("express-migrant-crisis.csv")

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

df = get_info(express_migrant_crisis)

# Rewrite to CSV
df.to_csv("express-migrant-crisis.csv", encoding="utf-8", header=True, index=False)


## Merging all the dataframes

In [9]:
import pandas as pd
# load csvs
e1 = pd.read_csv("express-immigration.csv")
e2 = pd.read_csv("express-migrant-crisis.csv")

# combine all dataframes
express = pd.concat([e1,e2])

# write to csv
express.to_csv(r"all-express.csv", encoding="utf-8", header=True, index=False)
